In [ ]:
pip install -U catboost layer

In [2]:
# !pip -q install git+https://github.com/eaedk/testing-zindi-package.git
# from zindi.user import Zindian
# USERNAME = "adetoromichael346@gmail.com" #@param {type : "string"}
# user = Zindian(username=USERNAME)
# user.select_a_challenge(reward='all', kind='competition', active='true')
# user.download_dataset(destination="dataset")

In [4]:
import layer
from layer.decorators import dataset,model, pip_requirements

In [5]:
layer.login()
layer.init("ai-squad-zindi-air-quality")

Please open the following link in your web browser. Once logged in, copy the code and paste it here.
https://app.layer.ai/oauth/authorize?response_type=code&code_challenge=YV6rPjNbGSuhOJP6Vvd52kJaF8g3QwfaeGawchwWLAY&code_challenge_method=S256&client_id=0STDdcnpK48P8A429EAAn93WNuLmViLR&redirect_uri=https://app.layer.ai/oauth/code&scope=offline_access&audience=https://app.layer.ai
Code: gRKAIhq7AUX3c2xUyX0ZbkG_cdujRGZtXytqGMj00Jb4L
Successfully logged into https://app.layer.ai


Your Layer project is here: https://app.layer.ai/mike_ade/ai-squad-zindi-air-quality

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# pd.options.display.max_columns = 999
# pd.options.display.max_rows = 999
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import LabelEncoder

In [26]:
sub = layer.get_dataset("zindi/air-quality/datasets/sample_submission").to_pandas()
test_df = layer.get_dataset("zindi/air-quality/datasets/test").to_pandas()
train_df = layer.get_dataset("zindi/air-quality/datasets/train").to_pandas()

col1 = ['ID','date']
col2 = ['device']
col3 = ['humidity', 'temp_mean']
location = ['site_latitude', 'site_longitude']
chem = [
        'SulphurDioxide_SO2_column_number_density',
      'SulphurDioxide_SO2_column_number_density_amf',
        'SulphurDioxide_SO2_column_number_density_15km',
      'CarbonMonoxide_CO_column_number_density',
        'NitrogenDioxide_NO2_column_number_density',
        'UvAerosolIndex_absorbing_aerosol_index',
        'Ozone_O3_column_number_density',
        'Cloud_cloud_fraction', 
        'Cloud_cloud_top_pressure',
]
target = ['pm2_5']
feat = col1 + col2 + col3 + location + chem
train_df = train_df[feat + target]
test_df = test_df[feat]
train_df = train_df.sort_values(['date', 'device']).reset_index(drop=True) 
test_df = test_df.sort_values(['date', 'device']).reset_index(drop=True)

for dataset in (train_df,test_df):
    dataset['date'] = pd.to_datetime(dataset['date'])
    dataset['Day'] = dataset.date.dt.day
    dataset['Month'] = dataset.date.dt.month
    dataset['Year'] = dataset.date.dt.year
    dataset['DayOfWeek'] = dataset.date.dt.dayofweek
    dataset['DayOfYear'] = dataset.date.dt.dayofyear
    dataset['Week'] = dataset.date.dt.weekofyear    
    dataset.set_index('date', inplace=True)

ID = test_df['ID']
test_df.drop('ID',inplace=True,axis=1)
train_df.drop('ID',inplace=True,axis=1)
def LAG(data,LagFeature,shift=1,NewFeatures=[]) :
    data[NewFeatures[0]]   = data[LagFeature]  - data[LagFeature].shift(shift)
    data[NewFeatures[1]]   = data[LagFeature].shift(shift)

num_feats = train_df.columns
num_feats = num_feats.drop(['DayOfWeek','Month','Day','pm2_5','temp_mean','humidity','site_longitude', 'site_latitude','device', 'Year', 'DayOfYear', 'Week'])

for feature in num_feats:
    LAG(train_df,LagFeature=f'{feature}',shift=1,NewFeatures=[f'{feature}_diff_Lag1',f'{feature}_Lag1'])
    LAG(test_df,LagFeature=f'{feature}',shift=1,NewFeatures=[f'{feature}_diff_Lag1',f'{feature}_Lag1'])

for dataset in (train_df,test_df):
    dataset['Year_Week'] = dataset['Year'].astype(str) + '-' + dataset['Week'].astype(str)
    dataset['Month_Day'] = dataset['Month'].astype(str) + '-' + dataset['Day'].astype(str)
    dataset['lat_lon'] = dataset['site_latitude'].astype(str) + '_' + dataset['site_longitude'].astype(str)

feats = ['Year_Week','Month_Day', 'device', 'lat_lon']

#Label Encoder
def label_enc(train_df, test_df, features):
    lbl_enc = LabelEncoder()
    full_data = pd.concat([train_df[features], test_df[features]],axis=0)
    for col in (features):
        # print(col)
        lbl_enc.fit(full_data[col].values)
        train_df[col] = lbl_enc.transform(train_df[col])
        test_df[col] = lbl_enc.transform(test_df[col])
    return train_df, test_df

train_df,test_df = label_enc(train_df,test_df,feats)

DevicePM2_5Mean = dict(train_df.groupby('device')['pm2_5'].mean())
DevicePM2_5Std = dict(train_df.groupby('device')['pm2_5'].std())
DevicePM2_5Min = dict(train_df.groupby('device')['pm2_5'].min())
DevicePM2_5Max = dict(train_df.groupby('device')['pm2_5'].max())

for dataset in (train_df,test_df):
    dataset['DevicePM2_5Mean'] = dataset['device'].map(DevicePM2_5Mean)
    dataset['DevicePM2_5Std'] = dataset['device'].map(DevicePM2_5Std)
    dataset['DevicePM2_5Min'] = dataset['device'].map(DevicePM2_5Min)
    dataset['DevicePM2_5Max'] = dataset['device'].map(DevicePM2_5Max)

data = pd.concat([train_df, test_df], axis = 0)
def Agg(Features):
    for dataset in (train_df,test_df):
        for Feature in Features:
            dataset[f'{Feature}_PerMonth'] = dataset['Month'].map(dict(data.groupby('Month')[Feature].mean()))
            dataset[f'{Feature}_PerWeek'] = dataset['Year_Week'].map(dict(data.groupby('Year_Week')[Feature].mean()))
            dataset[f'{Feature}_PerDay'] = dataset['Month_Day'].map(dict(data.groupby('Month_Day')[Feature].mean()))

            dataset[f'{Feature}_Month_std'] = dataset['Month'].map(dict(data.groupby('Month')[Feature].std()))
            dataset[f'{Feature}_Week_std'] = dataset['Year_Week'].map(dict(data.groupby('Year_Week')[Feature].std()))
            dataset[f'{Feature}_Day_std'] = dataset['Month_Day'].map(dict(data.groupby('Month_Day')[Feature].std()))

            dataset[f'{Feature}_Month_min'] = dataset['Month'].map(dict(data.groupby('Month')[Feature].min()))
            dataset[f'{Feature}_Week_min'] = dataset['Year_Week'].map(dict(data.groupby('Year_Week')[Feature].min()))
            dataset[f'{Feature}_Day_min'] = dataset['Month_Day'].map(dict(data.groupby('Month_Day')[Feature].min()))

            dataset[f'{Feature}_Month_max'] = dataset['Month'].map(dict(data.groupby('Month')[Feature].max()))
            dataset[f'{Feature}_Week_max'] = dataset['Year_Week'].map(dict(data.groupby('Year_Week')[Feature].max()))
            dataset[f'{Feature}_Day_max'] = dataset['Month_Day'].map(dict(data.groupby('Month_Day')[Feature].max()))
        
Agg(['temp_mean', 'humidity'])
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
train_df.drop(['Year_Week', 'Month_Day', 'site_longitude', 'site_latitude'],inplace=True,axis=1)
test_df.drop(['Year_Week', 'Month_Day', 'site_longitude', 'site_latitude'],inplace=True,axis=1)

Output()

Output()

Output()

In [28]:
@model("air_quality_model")
@pip_requirements(packages=["catboost"])
def train():
    from catboost import CatBoostRegressor
    from sklearn.compose import TransformedTargetRegressor
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import KFold
    from sklearn.decomposition import PCA
    import datetime
    import warnings
    warnings.simplefilter(action='ignore')
    
    X = train_df.drop('pm2_5', axis = 1)
    y = train_df.pm2_5

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1, shuffle = True, random_state = 42)

    cb = CatBoostRegressor(n_estimators = 6886, learning_rate = 0.028926897706232692, depth = 8, verbose = 0, random_state = 42, task_type = 'GPU')
    model = TransformedTargetRegressor(cb, func = np.log1p, inverse_func = np.expm1)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    error = mean_squared_error(y_test, pred)
    layer.log({'Mean Absolute Error':error})

    return model

In [29]:
train()

Output()

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=<catboost.core.CatBoostRegressor object at 0x7f22aa776290>)

In [31]:
my_model = layer.get_model("ai-squad-zindi-air-quality/models/air_model").get_train()

Output()

In [34]:
pred = my_model.predict(test_df)
submission = pd.DataFrame({"Id": ID ,"pm2_5": pred})
submission.to_csv('air_quality.csv', index = False)
submission.head(10)

,Id,pm2_5
date,,
2020-08-20,ID_M1T3S50WVB,76.117701
2020-08-20,ID_S9OZLWWLTX,69.543607
2020-08-20,ID_OC21YTIKX2,81.596124
2020-08-20,ID_BNZBP8KDYD,82.423418
2020-08-20,ID_ZS2RAN8HZT,68.914078
2020-08-20,ID_HSJA8I0X66,82.861366
2020-08-20,ID_3O2ZUT59OO,69.447400
2020-08-20,ID_LWMVVRDK4P,73.556583
2020-08-20,ID_JGEEJ9J0G7,77.157679
